In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt


In [2]:
print("test")

test


# Analyse van het aantal in zijn geheel

In [3]:
df = pd.read_excel('Totaal_gebruik.xlsx')
df.head()

,Year,Snorfiets,Bromfiets,Totaal
0,2019,716917,387331,1104248
1,2020,728644,388345,1116989
2,2021,763342,393991,1157333
3,2022,781353,404282,1185635
4,2023,761219,436996,1198215


In [4]:
df = df.rename(columns={
    "Year": "Year",
    "Snorfiets": "Snorfiets",
    "Bromfiets": "Bromfiets"
})[["Year", "Snorfiets", "Bromfiets"]]

long = df.melt(id_vars="Year",
                value_vars=["Snorfiets", "Bromfiets"],
                var_name="Serie", value_name="Aantal")


fig = px.line(
    long, x="Year", y="Aantal", color="Serie",
    markers=True,
    labels={"Year": "Jaar", "Aantal": "Aantal", "Serie": ""},
)
fig.update_traces(hovertemplate="Jaar %{x}<br> Aantal %{y:.0f}")
fig.update_layout(title="Aantal gebruikers — Bromfiets vs. Snorfiets", xaxis=dict(dtick=1))
fig.show()



# Analyse van het aantal per leeftijdsgroep

In [29]:
df0 = pd.read_csv('leeftijd_schoon.csv')
df0.head()

,Jaar,Aantal,Type,Leeftijdscategorie
0,2019,7852.0,Snorfiets,16 tot 18 jaar
1,2020,8019.0,Snorfiets,16 tot 18 jaar
2,2021,7524.0,Snorfiets,16 tot 18 jaar
3,2022,6988.0,Snorfiets,16 tot 18 jaar
4,2023,7685.0,Snorfiets,16 tot 18 jaar


In [52]:
df_aantal_leeftijd = pd.read_excel('Aantal per leeftijd.xlsx').rename(columns={"Unnamed: 0": "Age"})

df_aantal_leeftijd["Age"] = (
    df_aantal_leeftijd["Age"]
    .str.replace(" jaar of ouder", "", regex=False)
    .str.replace(" jaar", "", regex=False)
    .astype(int)
)

bins = [15, 18, 20, 25, 30, 40, 50, 60, 65, 120]  # 120 als bovengrens voor "65 jaar of ouder"
labels = [
    "16 tot 18 jaar",
    "18 tot 20 jaar",
    "20 tot 25 jaar",
    "25 tot 30 jaar",
    "30 tot 40 jaar",
    "40 tot 50 jaar",
    "50 tot 60 jaar",
    "60 tot 65 jaar",
    "65 jaar of ouder"
]

# Maak een nieuwe kolom met categorieën
df_aantal_leeftijd["Age categories"] = pd.cut(
    df_aantal_leeftijd["Age"],
    bins=bins,
    labels=labels,
    right=False  # [15,18) betekent 16 en 17 vallen in "16 tot 18 jaar"
)


year_cols = [col for col in df_aantal_leeftijd.columns if str(col).isdigit()]

df_leeftijd_per_categorie = (
    df_aantal_leeftijd
    .groupby("Age categories")[year_cols]   
    .sum()
    .reset_index()
)

df_leeftijd_per_categorie


/var/folders/mb/m61rwffd0nd5sjp246scnmpc0000gn/T/ipykernel_49559/2662064920.py:36: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,Age categories,2019,2020,2021,2022,2023,2024,2025
0,16 tot 18 jaar,413630,412569,405921,395658,393633,392116,396511
1,18 tot 20 jaar,434083,438012,432244,436973,437356,429914,426276
2,20 tot 25 jaar,1079925,1094585,1107486,1132885,1167263,1182940,1184576
3,25 tot 30 jaar,1121863,1138965,1132942,1134506,1154999,1172484,1184374
4,30 tot 40 jaar,2108199,2147931,2180575,2220450,2289762,2341054,2378213
5,40 tot 50 jaar,2261107,2208076,2166062,2132501,2124542,2125060,2136753
6,50 tot 60 jaar,2508388,2532418,2549688,2548563,2541679,2508947,2464708
7,60 tot 65 jaar,1096839,1117798,1137661,1158497,1181421,1201970,1220243
8,65 jaar of ouder,3317851,3396844,3461908,3529960,3605524,3681644,3760045


In [98]:
df_leeftijd_per_categorie = df_leeftijd_per_categorie.rename(columns={"Age categories": "Leeftijdscategorie"})

# Zet df_leeftijd_per_categorie om van breed naar lang formaat
df_pop_long = df_leeftijd_per_categorie.melt(
    id_vars="Leeftijdscategorie",
    var_name="Jaar",
    value_name="Bevolking"
)

# Zorg dat 'Jaar' numeriek is
df_pop_long["Jaar"] = df_pop_long["Jaar"].astype(int)

# Merge met df0 (voertuigdata)
df_merged = pd.merge(
    df0,
    df_pop_long,
    on=["Jaar", "Leeftijdscategorie"],
    how="left"
)

# Bereken het gestandaardiseerde aantal
df_merged["Standardized"] = df_merged["Aantal"] / df_merged["Bevolking"]*1000

df_merged["Type"] = df_merged["Type"].replace({
    "Snorfiets": "Moped 25 km/h",
    "Bromfiets": "Moped 45 km/h"
})

df_merged["Leeftijdscategorie"] = df_merged["Leeftijdscategorie"].replace({
    "16 tot 18 jaar" : "16 - 18 years",
    "18 tot 20 jaar" : "18 - 20 years",
    "20 tot 25 jaar" : "20 - 25 years",
    "25 tot 30 jaar" : "25 - 30 years",
    "30 tot 40 jaar" : "30 - 40 years",
    "40 tot 50 jaar" : "40 - 50 years",
    "50 tot 60 jaar" : "50 - 60 years",
    "60 tot 65 jaar" : "60 - 60 years",
    "65 jaar of ouder" : "65 years and older"
})
# Bekijk het resultaat


df_merged.rename(columns={"Jaar": "Year"}, inplace=True)

df_merged.tail(10)

,Year,Aantal,Type,Leeftijdscategorie,Bevolking,Standardized
116,2023,36950.0,Moped 45 km/h,60 - 60 years,1181421,31.275896
117,2024,41698.0,Moped 45 km/h,60 - 60 years,1201970,34.691382
118,2025,44646.0,Moped 45 km/h,60 - 60 years,1220243,36.587794
119,2019,44058.0,Moped 45 km/h,65 years and older,3317851,13.279077
120,2020,46712.0,Moped 45 km/h,65 years and older,3396844,13.751588
121,2021,48675.0,Moped 45 km/h,65 years and older,3461908,14.060166
122,2022,50807.0,Moped 45 km/h,65 years and older,3529960,14.393081
123,2023,54715.0,Moped 45 km/h,65 years and older,3605524,15.175325
124,2024,60582.0,Moped 45 km/h,65 years and older,3681644,16.455149
125,2025,65538.0,Moped 45 km/h,65 years and older,3760045,17.430111


In [99]:
df_merged = (df_merged.groupby(["Year", "Leeftijdscategorie", "Type"])["Standardized"].sum().reset_index())


fig = px.line(
    data_frame = df_merged,
    x="Year",
    y="Standardized",
    color="Leeftijdscategorie",
    facet_col="Type",                 
    markers=True,
    category_orders={"Type": ["Moped 25 km/h", "Moped 45 km/h"]},
    labels={"Standardized": "Number per 1000 inhabitants", "Aantal": "Aantal", "Leeftijdscategorie": "Age categories"},
    title="Number of mopeds per age categorie"
)


fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

# Y-as delen voor eerlijke vergelijking
fig.update_yaxes(matches="y")

fig.update_layout(height=500, width=1000, legend_title_text="Age categories")
fig.show()

# Analyse van het aantal per gemeente

In [11]:
df1 = pd.read_excel('Gemeentes.xlsx')
df1.head()

,Region,Year,Number,Type,Inhabitants
0,Gooise Meren,2019,1651,Snorfietsen,57715
1,Gooise Meren,2020,1628,Snorfietsen,58055
2,Gooise Meren,2021,1635,Snorfietsen,58524
3,Gooise Meren,2022,1612,Snorfietsen,58846
4,Gooise Meren,2023,1614,Snorfietsen,60341


In [27]:
df1_standardized = df1.copy()
df1_standardized["Number per 1000 inhabitants"] = round(df1["Number"]/ df1["Inhabitants "]*1000)

df1_standardized["Type"] = df1_standardized["Type"].replace({
    "Snorfietsen": "Moped 25 km/h",
    "Bromfietsen": "Moped 45 km/h"
})

df1_standardized.head()

,Region,Year,Number,Type,Inhabitants,Number per 1000 inhabitants
0,Gooise Meren,2019,1651,Moped 25 km/h,57715,29.0
1,Gooise Meren,2020,1628,Moped 25 km/h,58055,28.0
2,Gooise Meren,2021,1635,Moped 25 km/h,58524,28.0
3,Gooise Meren,2022,1612,Moped 25 km/h,58846,27.0
4,Gooise Meren,2023,1614,Moped 25 km/h,60341,27.0


In [80]:
df_jaar_region = (df1_standardized.groupby(["Year", "Region", "Type"])["Number per 1000 inhabitants"].sum().reset_index())


fig = px.line(
    df_jaar_region, x="Year", y="Number per 1000 inhabitants", color="Region",
    facet_col="Type",  
    markers=True,
    title="Number of mopeds per region"
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

# Y-as delen voor eerlijke vergelijking
fig.update_yaxes(matches="y")

fig.update_layout(height=500, width=1000, legend_title_text="Region")
fig.show()

# Analyse van het aantal doden

In [100]:
df2 = pd.read_excel('Doden.xlsx')

In [14]:
df2 = df2.rename(columns={
    "Perioden": "Year",
    "Totaal": "Total",
    "Brom/Snorfiets": "MopedDeaths"
})[["Year", "Total", "MopedDeaths"]]

long = df2.melt(id_vars="Year",
                value_vars=["Total", "MopedDeaths"],
                var_name="Serie", value_name="Aantal")


fig = px.line(
    long, x="Year", y="Aantal", color="Serie",
    markers=True,
    labels={"Year": "Jaar", "Aantal": "Aantal doden", "Serie": ""},
    category_orders={"Serie": ["Total", "MopedDeaths"]},
)
fig.update_traces(hovertemplate="Jaar %{x}<br>%{y:.0f} doden")
fig.update_layout(title="Verkeersdoden — Totaal vs. Brom/snorfiets", xaxis=dict(dtick=1))
fig.show()

In [15]:

df2 = df2.rename(columns={
    "Perioden": "Year",
    "Totaal": "Total",
    "Brom/Snorfiets": "MopedDeaths"
})[["Year", "Total", "MopedDeaths"]]

long = df2.melt(id_vars="Year",
                value_vars=["Total", "MopedDeaths"],
                var_name="Serie", value_name="Aantal")



# 2) Zorg dat Year/Aantal numeriek zijn
long["Year"] = pd.to_numeric(long["Year"], errors="coerce")
long["Aantal"] = pd.to_numeric(long["Aantal"], errors="coerce")

# 3) Periode-labels + gecombineerde groep
long["Periode"] = np.where(long["Year"] < 2023, "Voor helmplicht", "Na helmplicht")
long["Groep"] = long["Serie"] + " — " + long["Periode"]

# 4) Plot met OLS-trendlijnen per groep
fig = px.scatter(
    long,
    x="Year", y="Aantal",
    color="Groep",               # 4 groepen => 4 trendlijnen
    trendline="ols",
    trendline_scope="trace",     # per kleur-trace
    labels={"Year": "Jaar", "Aantal": "Aantal gebruikers"},
    title="Aantal doden — Bromfiets vs. Snorfiets (voor en na helmplicht)"
)
fig.add_vline(x=2023, line_dash="dash", line_color="red",
              annotation_text="Helmet law inplemented ", annotation_position="top right")


fig.update_layout(xaxis=dict(dtick=1))
fig.show()